In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [2]:
cc = CasosConfirmados()
# cc.read()
# cc.save()
cc.load()
notifica = Notifica()
# notifica.download_todas_notificacoes()
# notifica.save()
notifica.load()
print(cc.shape())
print(notifica.shape())

(586944, 10728, 432271, 143945)
(2314138, 1062658, 11033, 1238275, 2172)


In [3]:
casosc = cc.get_casos()
casosn = notifica.get_casos()
# casosn = casosn.loc[(casosn['duplicado']==0) | ((casosn['duplicado']==1) & (casosn['manter']==1))]
print(casosn.shape)

(2314138, 55)


In [4]:
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.shape

(27859, 23)

In [5]:
idx_casos_notifica = casosn[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.shape

(652090, 55)

In [6]:
#merge hash_resid
casos_hash_resid = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])]
print(f"casos_hash_resid {len(casos_hash_resid)}")
notifica_hash_resid = casosn.loc[(casosn['hash_resid'].isin(casosc['hash_resid']))]
print(f"notifica_hash_resid {len(notifica_hash_resid)}")
merge_hash_resid = pd.merge(casos_hash_resid,notifica_hash_resid[['hash_resid','id','manter']])
print(f"merge_hash_resid {len(merge_hash_resid)}")
merge_hash_resid = merge_hash_resid.drop_duplicates('ordem')
print(f"merge_hash_resid {len(merge_hash_resid)}\n")

#merge hash_atend
casos_hash_atend = casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])]
print(f"casos_hash_atend {len(casos_hash_atend)}")
notifica_hash_atend = casosn.loc[(casosn['hash_atend'].isin(casosc['hash_atend']))]
print(f"notifica_hash_atend {len(notifica_hash_atend)}")
merge_hash_atend = pd.merge(casos_hash_atend,notifica_hash_atend[['hash_atend','id','manter']])
print(f"merge_hash_atend {len(merge_hash_atend)}")
merge_hash_atend = merge_hash_atend.drop_duplicates('ordem')
print(f"merge_hash_atend {len(merge_hash_atend)}\n")

#merge hash_diag
casos_hash_diag = casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])]
print(f"casos_hash_diag {len(casos_hash_diag)}")
notifica_hash_diag = casosn.loc[(casosn['hash_diag'].isin(casosc['hash_diag']))]
print(f"notifica_hash_diag {len(notifica_hash_diag)}")
merge_hash_diag = pd.merge(casos_hash_diag,notifica_hash_diag[['hash_diag','id','manter']])
print(f"merge_hash_diag {len(merge_hash_diag)}")
merge_hash_diag = merge_hash_diag.drop_duplicates('ordem')
print(f"merge_hash_diag {len(merge_hash_diag)}\n")

#trasnforma essa lista de index em um conjunto, para deixar apenas os index unicos
idx_casos_confirmados = set(casos_hash_resid.index.tolist() + casos_hash_atend.index.tolist() + casos_hash_diag.index.tolist())
print(f"idx_casos_confirmados {len(idx_casos_confirmados)}")

#realiza a diferença de conjunto para pegar os index dos que não foram encotrados no notifica
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
print(f"casos_confirmados_nao_notifica {len(casos_confirmados_nao_notifica)}") ## aqui o numero total é 27910 casos não encontrados no notifica
len(idx_casos_confirmados) + len(casos_confirmados_nao_notifica)

casos_hash_resid 537647
notifica_hash_resid 632348
merge_hash_resid 632348
merge_hash_resid 537647

casos_hash_atend 507065
notifica_hash_atend 579311
merge_hash_atend 585685
merge_hash_atend 507065

casos_hash_diag 478964
notifica_hash_diag 481380
merge_hash_diag 487369
merge_hash_diag 478964

idx_casos_confirmados 559085
casos_confirmados_nao_notifica 27859


586944

In [38]:
merge_casos_notifica = pd.concat([merge_hash_resid,merge_hash_atend,merge_hash_diag,casos_confirmados_nao_notifica]).drop_duplicates('ordem').sort_values('ordem')

merge_casos_notifica['evolucao'] = 3
merge_casos_notifica.loc[merge_casos_notifica['obito']==1, 'evolucao'] = 2
merge_casos_notifica.loc[merge_casos_notifica['recuperado']==1, 'evolucao'] = 1

merge_casos_notifica.loc[merge_casos_notifica['obito']==1, 'data_evolucao'] = merge_casos_notifica.loc[merge_casos_notifica['obito']==1, 'data_obito'] 
merge_casos_notifica.loc[merge_casos_notifica['obito']==1, 'data_com_evolucao'] = merge_casos_notifica.loc[merge_casos_notifica['obito']==1, 'data_com_obito'] 

merge_casos_notifica = merge_casos_notifica[['ordem', 'identificacao', 'id', 'ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'evolucao', 'data_evolucao', 'data_com_evolucao']]

In [39]:
writer = pd.ExcelWriter("merge_casos_notifica.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

merge_casos_notifica.to_excel(writer,sheet_name='geral',index=None)
worksheet = writer.sheets["geral"]
auto_fit_columns(worksheet,merge_casos_notifica)

ativos.to_excel(writer,sheet_name='ativos',index=None)
writer.save()
writer.close()